In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
!pip install seaborn
import seaborn as sns
import numpy as np

In [2]:
base_casas = pd.read_csv('house_prices.csv')
base_casas

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [3]:
X = base_casas.iloc[:,3:19].values
X

array([[ 3.00000e+00,  1.00000e+00,  1.18000e+03, ...,  9.81780e+04,
         4.75112e+01, -1.22257e+02],
       [ 3.00000e+00,  2.25000e+00,  2.57000e+03, ...,  9.81250e+04,
         4.77210e+01, -1.22319e+02],
       [ 2.00000e+00,  1.00000e+00,  7.70000e+02, ...,  9.80280e+04,
         4.77379e+01, -1.22233e+02],
       ...,
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75944e+01, -1.22299e+02],
       [ 3.00000e+00,  2.50000e+00,  1.60000e+03, ...,  9.80270e+04,
         4.75345e+01, -1.22069e+02],
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75941e+01, -1.22299e+02]])

In [4]:
y = base_casas.iloc[:,2].values
y

array([221900., 538000., 180000., ..., 402101., 400000., 325000.])

In [5]:
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')

In [6]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,y, test_size=0.3, random_state=0)

In [7]:
import sagemaker
import boto3
from sagemaker import Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [8]:
session = sagemaker.Session()
bucket = 'curso-aws-sagemaker-1736262933'
subpasta_modelo = 'modelos/house-prices/linear-learner'
subpasta_dataset = 'datasets/house-prices'

In [9]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::879381239590:role/service-role/AmazonSageMaker-ExecutionRole-20250103T092221


In [10]:
import io
import sagemaker.amazon.common as smac

buffer = io.BytesIO() 
smac.write_numpy_to_dense_tensor(buffer, X_treinamento, y_treinamento)
buffer.seek(0)

0

In [11]:
import os  
key = 'houses-train-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key)).upload_fileobj(buffer)

In [12]:
os.path.join(subpasta_dataset, 'train', key)

'datasets/house-prices/train/houses-train-data'

In [13]:
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key)
print(s3_train_data)

s3://curso-aws-sagemaker-1736262933/datasets/house-prices/train/houses-train-data


In [14]:
output_location  = 's3://{}/{}/output'.format(bucket, subpasta_modelo )
print(output_location)

s3://curso-aws-sagemaker-1736262933/modelos/house-prices/linear-learner/output


In [15]:
container = sagemaker.image_uris.retrieve(framework = 'linear-learner', region = boto3.Session().region_name)

In [16]:
linear = sagemaker.estimator.Estimator(image_uri = container,
                                          role = role,
                                          instance_count = 1,
                                          instance_type = 'ml.m5.large',
                                          output_path = output_location,
                                          sagemaker_session = session)

In [17]:
linear.set_hyperparameters(feature_dim = 16,
                           predictor_type = 'regressor',
                           num_models = 16)

In [18]:
linear.fit({'train': s3_train_data})

INFO:sagemaker:Creating training-job with name: linear-learner-2025-01-08-22-43-34-973


2025-01-08 22:43:37 Starting - Starting the training job...
2025-01-08 22:43:51 Starting - Preparing the instances for training...
2025-01-08 22:44:26 Downloading - Downloading input data......
2025-01-08 22:45:12 Downloading - Downloading the training image.....Docker entrypoint called with argument(s): train
Running default environment configuration script
[01/08/2025 22:46:22 INFO 140541444355904] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_insensitivity': '0.01', 'huber_delta':

In [19]:
linear_regressor = linear.deploy(initial_instance_count=1, instance_type='ml.m5.large')

INFO:sagemaker:Creating model with name: linear-learner-2025-01-08-22-47-23-058
INFO:sagemaker:Creating endpoint-config with name linear-learner-2025-01-08-22-47-23-058
INFO:sagemaker:Creating endpoint with name linear-learner-2025-01-08-22-47-23-058


-------!

In [43]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
linear_regressor.serializer = CSVSerializer()
linear_regressor.deserializer = JSONDeserializer()

In [44]:
X_teste.shape

(6484, 16)

In [45]:
results = linear_regressor.predict(X_teste)
results

{'predictions': [{'score': 396376.75},
  {'score': 1486270.75},
  {'score': 541274.75},
  {'score': 569960.75},
  {'score': 995760.75},
  {'score': 388720.75},
  {'score': 413164.75},
  {'score': 364538.75},
  {'score': 560250.75},
  {'score': 1230268.75},
  {'score': 712066.75},
  {'score': 379250.75},
  {'score': 389246.75},
  {'score': 282980.75},
  {'score': 843118.75},
  {'score': 397792.75},
  {'score': 356472.75},
  {'score': 687804.75},
  {'score': 357958.75},
  {'score': 564384.75},
  {'score': 365180.75},
  {'score': 407144.75},
  {'score': 627132.75},
  {'score': 195086.75},
  {'score': 599176.75},
  {'score': 297760.75},
  {'score': 471046.75},
  {'score': 672654.75},
  {'score': 912678.75},
  {'score': 498276.75},
  {'score': 1411902.75},
  {'score': 459310.75},
  {'score': 469354.75},
  {'score': 909416.75},
  {'score': 859094.75},
  {'score': 403978.75},
  {'score': 299222.75},
  {'score': 272606.75},
  {'score': 503788.75},
  {'score': 162828.75},
  {'score': 413634.75}

from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_teste, previsores)
mse = mean_squared_error(y_teste, previsores)
print ("mae, mse")

In [50]:
linear_regressor.delete_model()
linear_regressor.delete_endpoint()

INFO:sagemaker:Deleting model with name: linear-learner-2025-01-08-22-47-23-058
INFO:sagemaker:Deleting endpoint configuration with name: linear-learner-2025-01-08-22-47-23-058
INFO:sagemaker:Deleting endpoint with name: linear-learner-2025-01-08-22-47-23-058


SyntaxError: invalid decimal literal (1743692628.py, line 1)